In [8]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import linprog, minimize_scalar
from mpl_toolkits import mplot3d

from functions import *

# Helper functions
Projections, initialisations and visualisations.

In [9]:
# bound = [(0, 1)]

# def ortho_projection(A):
#     U, _, VT = np.linalg.svd(A, full_matrices=True)
#     return U@VT

# def perm_projection(A, A_eq, b_eq):
#     G = -A.flatten()
#     n = len(A)
#     result = linprog(G, A_eq=A_eq, b_eq=b_eq, bounds=bound*(n**2))
#     return result.x.reshape(n, n)

# def equality_constraints(n):
#     A_eq = np.zeros((2 * n, n ** 2))
#     for i in range(n):
#         A_eq[i, i * n : (i + 1) * n] = 1  # Constraint 1: Sum of each row of S equals 1
#         A_eq[n + i, i::n] = 1  # Constraint 2: Sum of each column of S equals 1
#     b_eq = np.ones(2 * n)
#     return A_eq, b_eq

# def ortho_from_permutation(P, X, Y):
#     return ortho_projection(X.T@P@Y)

# def permutation_from_ortho(Q, X, Y, A_eq, b_eq):
#     return np.transpose(perm_projection(Y@Q.T@X.T, A_eq, b_eq))

# def loss(X, Y, P, Q):
#     return np.linalg.norm(X@Q - P@Y, ord='fro')

# def augment_PandQ(X, Y, P, A_eq, b_eq):
#     Q = ortho_from_permutation(P, X, Y)
#     P = permutation_from_ortho(Q, X, Y, A_eq, b_eq)
#     return P, Q

# def matrix_heatmap(M):
#     plt.imshow(M, cmap='hot')
#     plt.show()

# def initialise_XYPstarQstar(n, d, sigma, plot=True, seed=154):
#     np.random.seed(seed)
#     X = np.zeros((n, d))
#     Z = np.zeros((n, d))
#     for i in range(n):
#         xi = np.random.multivariate_normal(mean = np.zeros(d), cov = np.eye(d))
#         X[i,:] = xi
#         zi = np.random.multivariate_normal(mean = np.zeros(d), cov = sigma**2 * np.eye(d))
#         Z[i,:] = zi

#     # Sample orthogonal matrix:
#     Helper = np.random.randn(d, d)
#     Qstar, _ = scipy.linalg.qr(Helper)

#     # Sample permutation and convert to matrix
#     pistar = np.random.permutation(n)
#     Pstar = np.zeros((n,n))
#     for i in range(n):
#         for j in range(n):
#             if pistar[i] == j:
#                 Pstar[i,j] = 1
#     Y = Pstar.T@X@Qstar + Z
#     if plot:
#         fig = plt.figure()
#         ax = plt.axes(projection='3d')
#         ax.scatter3D(X[:,0], X[:,1], X[:,2], color="green");
#         ax.scatter3D(Y[:,0], Y[:,1], Y[:,2], color="blue");
#         plt.show()
#     return X, Y, Pstar, Qstar

# Optimisation algorithms

### Frank Wolfe

We need to use Frank-Wolfe for solving
$$
D^* = \underset{D \in \mathcal{D}_n}{\mathrm{argmin}} \, \Vert AD - DB \Vert_F^2
$$
where $A = XX^T$, $B = YY^\top$, and
$$
\mathcal{D}_n = \left\{ D \in \mathbb{R}^{n\times n} \, : \, \sum_{i = 1}^n D_{ij} = 1 \text{ and } \sum_{j = 1}^n D_{ij} = 1 \right\}.
$$

For this, define
$$
    f: \mathbb{R}^{n^2} \to \mathbb{R}, \, D = (d_{ij})_{i, j = 1}^n \, \mapsto \, \Vert AD - DB \Vert_F^2
$$
and realise that $f$ is differentiable as well as convex. Furthermore, the set $\mathcal{D}_n$ is convex and compact. We will generate a sequence $D_i$ converging to $D^*$. 

In [10]:
def f(D, X, Y):
    A = X.T@X
    B = Y.T@Y
    return np.linalg.norm(A@D - D@B, ord='fro')**2

def grad_f(D, X, Y): 
    A = X.T@X
    B = Y.T@Y
    return 2*(A.T@A@D
             - A@D@B
             - A@D@B.T
             + D@B@B.T)
def alt_grad_f(D, X, Y):
    A = X.T@X
    B = Y.T@Y
    return -2*A@D@B

In [11]:
def frank_wolfe(D_init, X, Y, step_max, stepsize, A_eq, b_eq, tol=0.1, verbose=1):
    if verbose: print("Running Frank-Wolfe with "+str(step_max)+ " steps...")
    D = D_init.copy()
    if step_max == 0: return D
    n = len(D)
    if verbose>0: print("Loss value before iterations = "+str(f(D, X, Y)))
    step = 1
    while True:
        if verbose>1:print("FW step no "+str(step))
        G = (grad_f(D, X, Y)).flatten()
        # linprog minimizes by default, so we want min gradf dot S
        result = linprog(G, A_eq=A_eq, b_eq=b_eq, bounds=bound*(n**2))
        S_hat = result.x.reshape(n, n)
        Direction = S_hat - D
#         # Line search
#         gamma = minimize_scalar(
#             lambda gamma:f(D + gamma*Direction), 
#             bounds=(0,1), 
#             method='bounded').x
        
        D_new = D + stepsize*Direction
#         D = D + (2/(step + 1))*Direction
        if step == step_max:
                if verbose>0: print("Reached max number of iterations.")
                if verbose>0: print("norm="+str(np.linalg.norm(D_new - D))+", grad_norm="+str(np.linalg.norm(grad_f(D_new, X, Y))))
                break
        D = D_new
        step += 1
    if verbose>0: print("Loss value after iterations = "+str(f(D, X, Y)))
    return D

In [19]:
def picard_curve(Pinit, Pstar, Qstar, X, Y, A_eq, b_eq, plot=True, verbose=False):
    ys = []
    qs = []
    ps = []
    my_xticks = []

    pindex = 0
    qindex = 0
    ps_turn = False
    n = len(X[0])
    d = len(X)
    lastP = np.zeros((n, n))
    lastQ = np.zeros((d, d))
    P = Pinit.copy()
    Q = np.ones((d, d))

    while np.linalg.norm(lastP-P) > 0 and np.linalg.norm(lastQ-Q) > 0:
        print(np.linalg.norm(lastP-P))
        if ps_turn:
            pindex += 1
            lastP = P
            P = permutation_from_ortho(Q, X, Y, A_eq, b_eq)
            ps_turn = False
        else:
            qindex += 1
            lastQ = Q
            Q = ortho_from_permutation(P, X, Y)
            ps_turn = True
        ys.append(loss(X, Y, P, Q))
        ps.append(np.linalg.norm(P - Pstar))
        qs.append(np.linalg.norm(Q - Qstar))
        my_xticks.append("(P"+str(pindex)+", Q"+str(qindex)+")")
    xs = range(len(ys))
    if plot:
        plt.plot(xs, ys, label="||XQ - PY||")
        plt.plot(xs, qs, label="||Q - Q*||")
        plt.plot(xs, ps, label="||P - P*||")
        plt.xticks(xs, my_xticks)
        plt.legend()
        plt.show()
    if verbose: print("Loss at (P*, Q*) equals: " + str(loss(X, Y, Pstar, Qstar)))
    return np.trace(P.T@Pstar)


# Experiments

Here, we try the above functions for different values of $n$, $d$ and $\sigma$. 

In [20]:
def setup_experiment(optim_function, n, d, sigma, D0_steps, stepsize):
    A_eq, b_eq = equality_constraints(n)
    X, Y, Pstar, Qstar = initialise_XYPstarQstar(n, d, sigma, plot = False)
    D_init = (1/n)*np.ones((n, n))
    D0 = optim_function(D_init, X, Y, D0_steps, stepsize, A_eq, b_eq, verbose = 0)
    return D0, X, Y, A_eq, b_eq, Pstar, Qstar

def evaluate_convergence(optim_function, n, d, sigma, D0_steps, stepsize):
    D0, X, Y, A_eq, b_eq, Pstar, Qstar = setup_experiment(optim_function, n, d, sigma, D0_steps, stepsize)
    overlap = picard_curve(D0, Pstar, Qstar, X, Y, A_eq, b_eq, plot = False)
    print(overlap + " overlap for "+str(optim_function.__name__)+" with n="+str(n)+", d="+str(d)
          +", sig="+str(sigma) +", D0_steps="+("0" if steps<10 else "")+str(steps)
          +", stepsize="+str(stepsize))
    

In [21]:
for sigma in [0.05, 0.1, 0.2]:
    for d in [10, 20, 50]:
        print("----- d="+str(d)+ ", sigma="+str(sigma)+" -----")
        for D0_steps in [1, 20, 100, 1000]:
            n = 100
            stepsize = 1/100
            evaluate_convergence(frank_wolfe, n, d, sigma, D0_steps, stepsize)
            


----- d=10, sigma=0.05 -----
0.9999999999999613
0.9999999999999613
9.949874371066697
9.949874371066697
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.92838827718412
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.92838827718412
13.92838827718412
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.784048752090222
13.784048752090222
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

...

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460

13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018
13.856406460551018


KeyboardInterrupt: 

In [ ]:
def Birkhoff_projection(M):
    # Projects matrix onto the closest doubly stochastic matrix
    ...

def projected_gradient_descent(D_init, X, Y, step_max, stepsize, A_eq, b_eq, tol=0.1, verbose=1):
    if verbose>0: print("Running Projected Gradient Descent with "+str(step_max)+ " iterations and step size "+str(stepsize))
    D = D_init.copy()
    if step_max == 0: return D
    n = len(D)
    if verbose>0: print("Loss value before iterations = "+str(f(D, X, Y)))
    step = 1
    while True:
        if verbose>1:print("Projected GD step no. "+str(step))
        G = grad_f(D, X, Y)
        D = D - stepsize*G
        
#         D = Birkhoff_projection(D, A_eq, b_eq)
        
        if step == step_max:
                if verbose>0: print("Reached max number of iterations.")
                break
        step += 1
    if verbose>0: print("Loss value after iterations = "+str(f(D, X, Y)))
    return D

In [ ]:
n=100
d=15
sigma=0.4
steps=50
stepsize=1/50

evaluate_convergence(frank_wolfe, n, d, sigma, steps, stepsize)
evaluate_convergence(projected_gradient_descent, n, d, sigma, steps, stepsize)

# Augmentation via belief propagation
This seems already better now. Let's try to augment the solution since we're super far from the true one.

Other Todos: 
- Reimplement the stuff from Berthet paper and see how we're doing
- Do the descent from Berthet and see how it compares

In [ ]:
tronc_lines = np.zeros((n, n))
for i in range(n):
    j = np.argmax(D[i, :])
    tronc_lines[i,j] = 1
    
plt.imshow(tronc_lines, cmap='hot')
plt.title("Keeping only maximising index of each line in D")
plt.show()

print("Loss trajectory when initialising with max line element")
P, Q = augment_PandQ(X, Y, tronc_lines, A_eq, b_eq)
print(loss(X, Y, P, Q))
for i in range(4):
    P, Q = augment_PandQ(X, Y, P, A_eq, b_eq)
    print(loss(X, Y, P, Q))
print("Loss at (P*, Q*) equals: " + str(loss(X, Y, Pstar, Qstar)))

# Archive, code not needed anymore

In [ ]:
print("Histograms for the doubly stochastic matrix")


vecD = D.flatten()
plt.figure(figsize=(8,5))
plt.hist(vecD, bins=50, rwidth = 0.7)
plt.title("histogram values in vecD, LOGSCALE")
plt.yscale("log")
plt.show()


n_plots = 5

plt.figure(figsize=(12, 18))
for i in range(n_plots):
    plt.subplot(n_plots, 2, 2*i+1)
    plt.hist(vecD[i*n:(i+1)*n], bins=50, rwidth = 0.7)
    plt.title("Histogram of LINE number " +str(i+1)+" of D")
    plt.yscale("log")
    
    plt.subplot(n_plots, 2, 2*i+2)
    plt.hist(vecD[i::n], bins=50, rwidth = 0.7, color="red")
    plt.title("Histogram of COLUMN number " +str(i+1)+" of D")
    plt.yscale("log")

plt.show()

In [ ]:
P0 = perm_projection(D, A_eq, b_eq)
P, Q = augment_PandQ(X, Y, P0, A_eq, b_eq)
print("Loss trajectory for initialisation at P0 = Proj_Perm(D)")
print(loss(X, Y, P, Q))
for i in range(6):
    P, Q = augment_PandQ(X, Y, P, A_eq, b_eq)
    print(loss(X, Y, P, Q))
print("Loss at (P*, Q*) equals: " + str(loss(X, Y, Pstar, Qstar)))

In [ ]:
# Some other matrices to check out the alternative optim algo.
pirand = np.random.permutation(n)
Prand = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if pirand[i] == j:
            Prand[i,j] = 1
            
PprojD = perm_projection(D, A_eq, b_eq)

D_tronc_lines = np.zeros((n, n))
for i in range(n):
    j = np.argmax(D[i, :])
    D_tronc_lines[i,j] = 1
print("Defined Prand, PprojD, D_tronc_lines")
picard_curve(Prand, X, Y, A_eq, b_eq)